In [10]:
!pip install git+https://github.com/christophM/rulefit.git

  Cloning https://github.com/christophM/rulefit.git to /tmp/pip-req-build-cszzc31n
  Running command git clone --filter=blob:none --quiet https://github.com/christophM/rulefit.git /tmp/pip-req-build-cszzc31n
  Resolved https://github.com/christophM/rulefit.git to commit 472b8574b4eb9e565caf1e05ed580998fe2c9a8e
  Preparing metadata (setup.py) ... done


In [11]:
import numpy as np
import pandas as pd

from rulefit import RuleFit, RuleEnsemble

from sklearn.preprocessing import LabelEncoder



In [12]:

carbon_data = pd.read_csv("res/carbon_emissions.csv", index_col=0)
carbon_data.head()

,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
Body Type,,,,,,,,,,,,,,,,,,,
overweight,female,pescatarian,daily,coal,public,NaN,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
obese,female,vegetarian,less frequently,natural gas,walk/bicycle,NaN,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595
overweight,male,omnivore,twice a day,wood,walk/bicycle,NaN,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,"['Paper', 'Plastic', 'Glass', 'Metal']","['Microwave', 'Grill', 'Airfryer']",1074
obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,['Paper'],['Oven'],4743


In [13]:
def get_encoded(label, data, encoder):
    le = LabelEncoder()
    le.fit(data[label])
    encoder[label] = le
    return le.transform(data[label])
    

In [14]:
# preprocessing of data:
#carbon_data = carbon_data.dropna(how='any') # drop Nan Values for now in a way i dont care

y = carbon_data.CarbonEmission.values

X = carbon_data.drop("CarbonEmission", axis=1)
#X = carbon_data[["Sex", "Diet"]]
features = X.columns
encoder_dict = {}
for label in X.columns:
    if X[label].dtype != "int" or X[label].dtype != "float":
        X[label] = get_encoded(label, X, encoder_dict)


X = X.to_numpy()
X.shape


(10000, 18)

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=500, max_depth=10, learning_rate=0.01)
rf = RuleFit(tree_generator=gb, max_iter=1000)

rf.fit(X, y, feature_names=features)

/home/robin/anaconda3/envs/uu_courses/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.456e+06, tolerance: 1.036e+06
  model = cd_fast.enet_coordinate_descent(


RuleFit(max_iter=1000,
        tree_generator=GradientBoostingRegressor(learning_rate=0.01,
                                                 max_depth=10, max_leaf_nodes=2,
                                                 n_estimators=554,
                                                 random_state=553))

In [16]:
rf.predict(X)

array([2072.87729873, 1456.24833325, 2301.67931226, ..., 2391.15164361,
       4900.86411716, 1154.38923984])

In [17]:

re = RuleEnsemble(rf.tree_generator)

rules = re.rules
#rules = rules[rules.coef != 0].sort_values("support", ascending=False)

print(rules)

[9 <= 2205.5 & 8 <= 2.5, 9 <= 2205.5 & 8 > 2.5, 9 > 2205.5, 9 <= 2063.5, 9 > 2063.5, 9 <= 2205.5 & 8 <= 2.5 & 8 <= 0.5, 9 <= 2205.5 & 8 <= 2.5 & 8 > 0.5, 9 > 2205.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 2205.5 & 5 <= 2.5 & 5 > 0.5 & 5 <= 1.5, 9 > 2205.5 & 5 <= 2.5 & 5 > 0.5 & 5 > 1.5, 9 > 2205.5 & 5 > 2.5, 9 <= 2205.5, 9 <= 2063.5 & 8 <= 2.5, 9 <= 2063.5 & 8 > 2.5, 9 > 2063.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 2063.5 & 5 <= 2.5 & 5 > 0.5, 9 > 2063.5 & 5 > 2.5, 9 <= 2063.5 & 8 <= 2.5 & 8 <= 0.5, 9 <= 2063.5 & 8 <= 2.5 & 8 > 0.5, 9 <= 2031.5 & 8 <= 2.5, 9 <= 2031.5 & 8 > 2.5, 9 > 2031.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 2031.5 & 5 <= 2.5 & 5 > 0.5, 9 > 2031.5 & 5 > 2.5, 9 <= 2031.5, 9 > 2031.5, 9 <= 2370.5 & 8 <= 2.5 & 8 <= 0.5, 9 <= 2370.5 & 8 <= 2.5 & 8 > 0.5, 9 <= 2370.5 & 8 > 2.5, 9 > 2370.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 2370.5 & 5 <= 2.5 & 5 > 0.5 & 5 <= 1.5, 9 > 2370.5 & 5 <= 2.5 & 5 > 0.5 & 5 > 1.5, 9 > 2370.5 & 5 > 2.5, 9 <= 2031.5 & 8 <= 2.5 & 8 <= 0.5, 9 <= 2031.5 & 8 <= 2.5 & 8 > 0.5, 9 <= 2448.5, 9 > 244